In [1]:
from math import radians, sin, cos, asin, sqrt
# This line imports specific functions from the math library that are used for calculating the haversine distance between two points on a globe.

import pandas as pd
# This line imports the Pandas library, which is a popular library for data manipulation and analysis.

from sklearn.preprocessing import LabelEncoder, StandardScaler
# This line imports the LabelEncoder and StandardScaler classes from the scikit-learn library, which are used for preprocessing data for machine learning models.

import time
# This line imports the time module, which is used for timing the execution of code

import numpy as np
# This line imports the NumPy library, which provides support for numerical computing with Python.

In [3]:
orders = pd.read_csv("./data/olist_orders_dataset.csv")
# This line reads the CSV file "olist_orders_dataset.csv" located in the "./data/" directory and stores its contents as a Pandas DataFrame named "orders".

items = pd.read_csv("./data/olist_order_items_dataset.csv")
# This line reads the CSV file "olist_order_items_dataset.csv" located in the "./data/" directory and stores its contents as a Pandas DataFrame named "items".

customers = pd.read_csv("./data/olist_customers_dataset.csv")
# This line reads the CSV file "olist_customers_dataset.csv" located in the "./data/" directory and stores its contents as a Pandas DataFrame named "customers".

sellers = pd.read_csv("./data/olist_sellers_dataset.csv")
# This line reads the CSV file "olist_sellers_dataset.csv" located in the "./data/" directory and stores its contents as a Pandas DataFrame named "sellers".

geo = pd.read_csv("./data/olist_geolocation_dataset.csv")
# This line reads the CSV file "olist_geolocation_dataset.csv" located in the "./data/" directory and stores its contents as a Pandas DataFrame named "geo".

products = pd.read_csv("./data/olist_products_dataset.csv")
# This line reads the CSV file "olist_products_dataset.csv" located in the "./data/" directory and stores its contents as a Pandas DataFrame named "products".

In [5]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [7]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [9]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [11]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [13]:
products.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [15]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [17]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [19]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [21]:
# Get the seller zip code of each order

middle = items[['order_id', 'seller_id']]
# This line creates a new dataframe middle that contains only the order_id and seller_id columns from the items dataframe.

middle_2 = middle.merge(sellers[['seller_id', 'seller_zip_code_prefix']], on="seller_id", how="outer")
# This line merges the middle dataframe with the seller_id and seller_zip_code_prefix columns from the sellers dataframe, creating a new dataframe middle_2. The outer join type is used, which means that all rows from both dataframes are included in the merged dataframe, and missing values are filled with NaN.

orders = orders.merge(middle_2, on="order_id", how="left")
# This line merges the orders dataframe with the middle_2 dataframe on the order_id column, creating a new orders dataframe. The left join type is used, which means that all rows from the orders dataframe are included in the merged dataframe, and missing values from the middle_2 dataframe are filled with NaN.

orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_id,seller_zip_code_prefix
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3504c0cb71d7fa48d967e0e4c94d59d9,9350.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,289cdb325fb7e7f891c38608bf9e0962,31570.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,4869f7a5dfa277a7dca6462dcf3b52b2,14840.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,66922902710d126a0e7d26b0e3805106,31842.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2c9e548be18521d1c43cde1c582c6de8,8752.0


In [23]:
# Get customer zip code of each order
orders = orders.merge(customers[['customer_id', 'customer_zip_code_prefix']],
                  on='customer_id', how="left")
# The code above performs a left merge operation between two Pandas dataframes named "orders" and "customers" using the "customer_id" column as the joining key. It then selects the "customer_id" and "customer_zip_code_prefix" columns from the "customers" dataframe and merges them with the "orders" dataframe based on the matching "customer_id" column.


orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_id,seller_zip_code_prefix,customer_zip_code_prefix
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3504c0cb71d7fa48d967e0e4c94d59d9,9350.0,3149
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,289cdb325fb7e7f891c38608bf9e0962,31570.0,47813
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,4869f7a5dfa277a7dca6462dcf3b52b2,14840.0,75265
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,66922902710d126a0e7d26b0e3805106,31842.0,59296
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2c9e548be18521d1c43cde1c582c6de8,8752.0,9195


In [25]:
# Clean geo df
geo = geo[~geo['geolocation_zip_code_prefix'].duplicated()]
'''
This line first selects the 'geolocation_zip_code_prefix' column from the 'geo' DataFrame using the indexing operator []. The duplicated() method is then called on this column to create a boolean mask of rows that have duplicate values in this column.

The tilde operator (~) is used to invert this boolean mask, so that the mask contains True for rows that do not have duplicate values in this column.
'''

geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
5,1012,-23.547762,-46.635361,são paulo,SP


In [27]:
# add seller coordinates to the orders
orders = orders.merge(geo, left_on="seller_zip_code_prefix",
                      right_on="geolocation_zip_code_prefix", how="left")
'''
The code above performs a left join of two dataframes - "orders" and "geo" - using the "seller_zip_code_prefix" column of the "orders" dataframe and the "geolocation_zip_code_prefix" column of the "geo" dataframe as the join keys.
'''
                      
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_id,seller_zip_code_prefix,customer_zip_code_prefix,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3504c0cb71d7fa48d967e0e4c94d59d9,9350.0,3149,9350.0,-23.680114,-46.452454,maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,289cdb325fb7e7f891c38608bf9e0962,31570.0,47813,31570.0,-19.810119,-43.984727,belo horizonte,MG
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,4869f7a5dfa277a7dca6462dcf3b52b2,14840.0,75265,14840.0,-21.362358,-48.232976,guariba,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,66922902710d126a0e7d26b0e3805106,31842.0,59296,31842.0,-19.840168,-43.923299,belo horizonte,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2c9e548be18521d1c43cde1c582c6de8,8752.0,9195,8752.0,-23.551707,-46.260979,mogi das cruzes,SP


In [31]:
# add customer coordinates to the orders
orders = orders.merge(geo, left_on="customer_zip_code_prefix",
                      right_on="geolocation_zip_code_prefix", how="left",
                      suffixes=("_seller", "_customer"))
# This code merges two Pandas DataFrames, orders and geo, on a common column, customer_zip_code_prefix in orders and geolocation_zip_code_prefix in geo. The resulting merged DataFrame contains all the columns from both DataFrames.

In [33]:
# Clean orders
# 1-Filter out orders with multiple sellers Because each order only has one delivery date
df = orders.groupby(by="order_id").nunique()
# Groups the orders by order_id and calculates the number of unique values in each group using the nunique() method.

mono_orders = pd.Series(df[df['seller_id'] == 1].index)
# Selects the indices of the resulting DataFrame where the seller_id column equals 1 and stores them in a Pandas Series called mono_orders.

filtered_orders = orders.merge(mono_orders, how='inner')
# Merges the original orders DataFrame with mono_orders based on the order_id column using an inner join and stores the resulting DataFrame in a variable called filtered_orders.

In [35]:
# 2-drop rows with missing values
filtered_orders = filtered_orders.drop(columns=["order_approved_at"])
# This line drops the "order_approved_at" column from the filtered_orders DataFrame. This column is not necessary for the analysis being performed.

filtered_orders = filtered_orders.dropna()
# This line drops any rows in the filtered_orders DataFrame that contain missing values. This is a common data preprocessing step to ensure that the dataset is clean and complete.


filtered_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_id,seller_zip_code_prefix,customer_zip_code_prefix,...,geolocation_zip_code_prefix_y,geolocation_lat_y,geolocation_lng_y,geolocation_city_y,geolocation_state_y,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3504c0cb71d7fa48d967e0e4c94d59d9,9350.0,3149,...,9350.0,-23.680114,-46.452454,maua,SP,3149.0,-23.574809,-46.587471,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,289cdb325fb7e7f891c38608bf9e0962,31570.0,47813,...,31570.0,-19.810119,-43.984727,belo horizonte,MG,47813.0,-12.169860,-44.988369,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,4869f7a5dfa277a7dca6462dcf3b52b2,14840.0,75265,...,14840.0,-21.362358,-48.232976,guariba,SP,75265.0,-16.746337,-48.514624,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,66922902710d126a0e7d26b0e3805106,31842.0,59296,...,31842.0,-19.840168,-43.923299,belo horizonte,MG,59296.0,-5.767733,-35.275467,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2c9e548be18521d1c43cde1c582c6de8,8752.0,9195,...,8752.0,-23.551707,-46.260979,mogi das cruzes,SP,9195.0,-23.675037,-46.524784,santo andre,SP


In [37]:
# Define Function to calculate distance
def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Compute distance between two pairs of (lat, lng)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

# This line defines a function called haversine_distance that takes four arguments: lon1, lat1, lon2, and lat2. These arguments represent the longitude and latitude coordinates of two points on the globe.

In [39]:
def get_package_size(items, products):
    # Get Package Size
    df_tmp = items[['order_id', 'product_id']].merge(products[['product_id', 'product_length_cm', 'product_height_cm',
                                                               'product_width_cm', 'product_weight_g']],
                                                     on="product_id",
                                                     how="outer")
    df_tmp.loc[:, "product_size_cm3"] = \
        df_tmp['product_length_cm']*df_tmp['product_width_cm'] * df_tmp['product_height_cm']
    orders_size_weight = df_tmp.groupby("order_id", as_index=False).sum()[['order_id', 'product_size_cm3',
                                                                           'product_weight_g']]
    return orders_size_weight

# The code above defines a function called get_package_size that takes two arguments: items and products, which are Pandas DataFrames containing information about products and the items ordered.

In [41]:
def object_to_int(dataframe_series):

    if dataframe_series.dtype == 'object':
        # This line checks if the data type of the input dataframe_series is "object", which typically represents string or categorical data.

        dataframe_series = LabelEncoder().fit_transform(dataframe_series)
        # This line uses the LabelEncoder() method from the scikit-learn library to encode the string or categorical data as integers. This is a common preprocessing step in machine learning to convert non-numeric data into a format that can be used by algorithms. The fit_transform() method fits the encoder to the data and transforms it.
        
    return dataframe_series

In [43]:
#It gives the numerical statistical information of the dataframe
orders.describe()

'''
The code above applies the describe() method to a Pandas DataFrame called orders, which provides summary statistics of the data in the DataFrame. Here's a brief explanation of what each statistic means:

count: the number of non-missing values in each column
mean: the average value of each column
std: the standard deviation of each column
min: the minimum value of each column
25%: the first quartile of each column (25th percentile)
50%: the second quartile of each column (50th percentile, equivalent to the median)
75%: the third quartile of each column (75th percentile)
max: the maximum value of each column
The describe() method is useful for quickly getting an overview of the data in a DataFrame, including identifying potential outliers or unusual patterns in the data.
'''

"\nThe code above applies the describe() method to a Pandas DataFrame called orders, which provides summary statistics of the data in the DataFrame. Here's a brief explanation of what each statistic means:\n\ncount: the number of non-missing values in each column\nmean: the average value of each column\nstd: the standard deviation of each column\nmin: the minimum value of each column\n25%: the first quartile of each column (25th percentile)\n50%: the second quartile of each column (50th percentile, equivalent to the median)\n75%: the third quartile of each column (75th percentile)\nmax: the maximum value of each column\nThe describe() method is useful for quickly getting an overview of the data in a DataFrame, including identifying potential outliers or unusual patterns in the data.\n"

In [45]:
# It returns the number of null values in the dataframe for every column feature
# Using info() we can view the number of non-null values whereas isnull() gives the number of null values
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 161
order_delivered_carrier_date     1968
order_delivered_customer_date    3229
order_estimated_delivery_date       0
seller_id                         775
seller_zip_code_prefix            775
customer_zip_code_prefix            0
geolocation_zip_code_prefix_x    1028
geolocation_lat_x                1028
geolocation_lng_x                1028
geolocation_city_x               1028
geolocation_state_x              1028
geolocation_zip_code_prefix_y    1028
geolocation_lat_y                1028
geolocation_lng_y                1028
geolocation_city_y               1028
geolocation_state_y              1028
geolocation_zip_code_prefix       306
geolocation_lat                   306
geolocation_lng                   306
geolocation_city                  306
geolocation_state                 306
dtype: int64

In [47]:
def scaling(X_train, X_test):
    '''
    The code above defines a function called scaling that performs feature scaling using the StandardScaler class from scikit-learn.
    '''
    sc_X = StandardScaler()
    X_train_scaled = sc_X.fit_transform(X_train)
    X_test_scaled = sc_X.fit_transform(X_test)
    return X_train_scaled, X_test_scaled

In [49]:
def split_data(final_df, columns_for_train, columns_for_pred, i_flag):
    '''
    This function is a convenient way to split a DataFrame into training and testing sets for machine learning purposes. The i_flag parameter is used to avoid a warning message related to the NumPy library, and the columns_for_train and columns_for_pred parameters allow for flexible selection of features and target variables.
    '''

    if i_flag:
        from sklearnex import patch_sklearn  # pylint: disable=C0415, E0401
        patch_sklearn()
    from sklearn.model_selection import train_test_split  # pylint: disable=C0415

    x_train, x_test, y_train, y_test = train_test_split(final_df[columns_for_train], final_df[columns_for_pred],
                                                        test_size=0.3, random_state=42)
    return x_train, x_test, y_train, y_test

In [77]:
# Assuming haversine_distance, get_package_size, object_to_int are defined elsewhere
# Assuming filtered_orders is in the state shown by your .info() output

print("--- Initial State Check ---")
print(f"Columns: {filtered_orders.columns.tolist()}")
print(f"Shape: {filtered_orders.shape}")

# --- Step 1: Clean up stale/incorrect columns from previous failed attempts ---
print("\n--- Step 1: Dropping stale calculation columns ---")
cols_to_drop = ['wait_time', 'est_wait_time', 'purchase_dow', 'year', 'purchase_month', 'distance']
# Check which ones actually exist before dropping
cols_exist_to_drop = [col for col in cols_to_drop if col in filtered_orders.columns]
if cols_exist_to_drop:
    filtered_orders = filtered_orders.drop(columns=cols_exist_to_drop)
    print(f"Dropped: {cols_exist_to_drop}")
else:
    print("No stale calculation columns found to drop.")

# --- Step 2: Resolve Product Column Suffixes ---
print("\n--- Step 2: Resolving product column suffixes ---")
# IMPORTANT: Assuming the '_x' columns are the correct ones you want to keep.
# Verify this assumption based on your data preparation steps!
product_rename_map = {
    'product_size_cm3_x': 'product_size_cm3',
    'product_weight_g_x': 'product_weight_g'
}
product_cols_to_drop = ['product_size_cm3_y', 'product_weight_g_y']

# Check if _x columns exist for renaming
product_cols_to_rename = {k: v for k, v in product_rename_map.items() if k in filtered_orders.columns}
if product_cols_to_rename:
    filtered_orders = filtered_orders.rename(columns=product_cols_to_rename)
    print(f"Renamed product columns: {product_cols_to_rename}")

    # Drop the other set (_y columns)
    product_cols_exist_to_drop = [col for col in product_cols_to_drop if col in filtered_orders.columns]
    if product_cols_exist_to_drop:
        filtered_orders = filtered_orders.drop(columns=product_cols_exist_to_drop)
        print(f"Dropped unused product columns: {product_cols_exist_to_drop}")
else:
     # Check if columns already exist without suffix (maybe fixed earlier?)
     if 'product_size_cm3' in filtered_orders.columns and 'product_weight_g' in filtered_orders.columns:
         print("Product columns seem to already have correct names (no _x/_y suffix).")
     else:
         print("Warning: Could not find product columns with _x suffix to rename.")


# --- Step 3: Rename Geolocation State Columns ---
print("\n--- Step 3: Renaming geolocation state columns ---")
# Assuming _x = seller, _y = customer based on common patterns
geo_state_rename_map = {
    'geolocation_state_x': 'geolocation_state_seller',
    'geolocation_state_y': 'geolocation_state_customer'
}
# Check if _x/_y columns exist for renaming
geo_cols_to_rename = {k: v for k, v in geo_state_rename_map.items() if k in filtered_orders.columns}
if geo_cols_to_rename:
    filtered_orders = filtered_orders.rename(columns=geo_cols_to_rename)
    print(f"Renamed geolocation state columns: {geo_cols_to_rename}")
else:
    # Check if columns already exist with seller/customer names
     if 'geolocation_state_seller' in filtered_orders.columns and 'geolocation_state_customer' in filtered_orders.columns:
         print("Geolocation state columns seem to already have correct names (seller/customer).")
     else:
        print("Warning: Could not find geolocation state columns with _x/_y suffix to rename.")


# ========== NOW START THE MAIN PROCESSING BLOCK ==========
# The DataFrame should now be cleaned up and ready for calculations.

print("\n--- Starting Main Processing Block ---")

# --- Calculate Haversine distance ---
# Uses _x/_y directly as state columns were renamed, but lat/lon weren't necessarily
print("Calculating Haversine distance...")
required_dist_cols = ['geolocation_lng_x', 'geolocation_lat_x', 'geolocation_lng_y', 'geolocation_lat_y']
if all(col in filtered_orders.columns for col in required_dist_cols):
    filtered_orders['distance'] = filtered_orders.apply(
        lambda row: haversine_distance(
            row["geolocation_lng_x"], row["geolocation_lat_x"],
            row["geolocation_lng_y"], row["geolocation_lat_y"]
        ),
        axis=1,
    )
    print("Distance calculation complete.")
else:
    print(f"Warning: Could not calculate distance. Missing one or more columns from: {required_dist_cols}")
    filtered_orders['distance'] = np.nan


# --- Merge Package Info ---
# NOTE: This seems redundant if product columns were already present with _x/_y suffixes.
# You might need to revisit if/how this merge should happen.
# If product info came from an earlier merge, you might skip this.
# print("Merging package size/weight...")
# orders_size_weight = get_package_size(items, products)
# filtered_orders = filtered_orders.merge(orders_size_weight, on='order_id', how='left')
# print("Package info merge complete.")
# --> Skipping this merge for now, assuming product info is already present from the _x cols


# --- Process time columns ---
print("Converting time columns...")
time_columns = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
conversion_successful = True
for column in time_columns:
    if column in filtered_orders.columns:
        # Ensure it's not already datetime64
        if not pd.api.types.is_datetime64_any_dtype(filtered_orders[column]):
             filtered_orders.loc[:, column] = pd.to_datetime(filtered_orders[column], errors='coerce')
             if filtered_orders[column].isna().all():
                 print(f"Warning: Column '{column}' became all NaT after conversion.")
                 conversion_successful = False
        else:
             print(f"Column '{column}' is already datetime type.")
    else:
        print(f"Warning: Time column '{column}' not found.")
        conversion_successful = False
if conversion_successful:
    print("Time column conversion complete.")
else:
    print("Time column conversion may have failed or columns were missing.")


# --- Calculate Wait Times ---
print("Calculating wait times...")
if ('order_delivered_customer_date' in filtered_orders.columns and
    'order_purchase_timestamp' in filtered_orders.columns and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_delivered_customer_date']) and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_purchase_timestamp'])):
    filtered_orders.loc[:, "wait_time"] = (filtered_orders['order_delivered_customer_date'] - filtered_orders['order_purchase_timestamp']).dt.days
else:
    print("Warning: Cannot calculate actual wait time.")
    if "wait_time" not in filtered_orders.columns: filtered_orders.loc[:, "wait_time"] = np.nan

if ('order_estimated_delivery_date' in filtered_orders.columns and
    'order_purchase_timestamp' in filtered_orders.columns and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_estimated_delivery_date']) and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_purchase_timestamp'])):
    filtered_orders.loc[:, "est_wait_time"] = (filtered_orders['order_estimated_delivery_date'] - filtered_orders['order_purchase_timestamp']).dt.days
else:
    print("Warning: Cannot calculate estimated wait time.")
    if "est_wait_time" not in filtered_orders.columns: filtered_orders.loc[:, "est_wait_time"] = np.nan


# --- Extract Date Components ---
print("Extracting date components...")
if ('order_purchase_timestamp' in filtered_orders.columns and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_purchase_timestamp'])):
    filtered_orders.loc[:, "purchase_dow"] = filtered_orders.order_purchase_timestamp.dt.dayofweek
    filtered_orders.loc[:, "year"] = filtered_orders.order_purchase_timestamp.dt.year
    filtered_orders.loc[:, "purchase_month"] = filtered_orders.order_purchase_timestamp.dt.month
else:
    print("Warning: Cannot extract date components.")
    if "purchase_dow" not in filtered_orders.columns: filtered_orders.loc[:, "purchase_dow"] = np.nan
    if "year" not in filtered_orders.columns: filtered_orders.loc[:, "year"] = np.nan
    if "purchase_month" not in filtered_orders.columns: filtered_orders.loc[:, "purchase_month"] = np.nan


# --- Create Final DataFrame ---
print("Creating final DataFrame...")
# Use the cleaned/renamed column names
final_columns_needed = [
    'purchase_dow', 'purchase_month', 'year',
    'product_size_cm3', 'product_weight_g', # Expecting these after renaming _x
    'geolocation_state_customer', # Expecting these after renaming _y
    'geolocation_state_seller',   # Expecting these after renaming _x
    'distance',
    'wait_time', 'est_wait_time'
]
final_columns_present = [col for col in final_columns_needed if col in filtered_orders.columns]
missing_final_cols = set(final_columns_needed) - set(final_columns_present)
if missing_final_cols:
    print(f"Warning: Columns still missing for final_df: {missing_final_cols}")

final_df = filtered_orders[final_columns_present].copy()


# --- Calculate Delay Feature ---
print("Calculating delay feature...")
if 'wait_time' in final_df.columns and 'est_wait_time' in final_df.columns:
    delay_diff = (final_df['wait_time'] - final_df['est_wait_time']).fillna(0)
    final_df['delay'] = (delay_diff > 0).astype(int)
else:
    print("Warning: Cannot calculate delay feature.")
    final_df['delay'] = 0


# --- Encode Final DataFrame ---
print("Encoding object columns...")
final_df_enc = final_df.apply(lambda x: object_to_int(x))


# --- Display results ---
print("\nFirst 5 rows of the final encoded DataFrame (final_df_enc):")
print(final_df_enc.head())
print("\nFinal DataFrame Info:")
print(final_df_enc.info())
print("\nScript finished.")

--- Initial State Check ---
Columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'seller_id', 'seller_zip_code_prefix', 'customer_zip_code_prefix', 'geolocation_zip_code_prefix_x', 'geolocation_lat_x', 'geolocation_lng_x', 'geolocation_city_x', 'geolocation_state_x', 'geolocation_zip_code_prefix_y', 'geolocation_lat_y', 'geolocation_lng_y', 'geolocation_city_y', 'geolocation_state_y', 'geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state', 'distance', 'product_size_cm3_x', 'product_weight_g_x', 'product_size_cm3_y', 'product_weight_g_y', 'wait_time', 'est_wait_time', 'purchase_dow', 'year', 'purchase_month']
Shape: (106578, 35)

--- Step 1: Dropping stale calculation columns ---
Dropped: ['wait_time', 'est_wait_time', 'purchase_dow', 'year', 'purchase_month', 'distance']

--- Step 2: Resolving product column 

In [79]:
# [Previous code: Initial state check, Steps 1, 2, 3]
# ... (Cleanup steps as before) ...

print("\n--- Starting Main Processing Block ---")

# ... (Distance Calculation as before) ...

# --- Process time columns ---
print("Converting time columns...")
time_columns = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
conversion_successful = True
for column in time_columns:
    if column in filtered_orders.columns:
        if not pd.api.types.is_datetime64_any_dtype(filtered_orders[column]):
             filtered_orders.loc[:, column] = pd.to_datetime(filtered_orders[column], errors='coerce')
             # Check right after conversion if ALL values became NaT
             if filtered_orders[column].isna().all():
                 print(f"----> CRITICAL WARNING: Column '{column}' is ALL NaT values after conversion!")
                 conversion_successful = False
             elif filtered_orders[column].isna().any():
                 print(f"----> NOTE: Column '{column}' has SOME NaT values after conversion.")
        else:
             print(f"Column '{column}' is already datetime type.")
    else:
        print(f"Warning: Time column '{column}' not found.")
        conversion_successful = False

if conversion_successful:
    print("Time column conversion attempt finished.")
else:
    print("Time column conversion may have failed or columns were missing/invalid.")

# ***** ADD THIS DEBUGGING BLOCK *****
print("\n--- Checking Time Columns AFTER Conversion ---")
if all(col in filtered_orders.columns for col in time_columns):
    print(filtered_orders[time_columns].info())
    print("\nHead of time columns:")
    print(filtered_orders[time_columns].head())
    print("\nNull/NaT counts in time columns:")
    print(filtered_orders[time_columns].isna().sum())
else:
    print("One or more time columns are missing, cannot debug their state.")
# ***********************************

# --- Calculate Wait Times ---
# ... (rest of the code remains the same) ...
# ... (wait time calc, date components, final_df, delay, encode, print head/info) ...


--- Starting Main Processing Block ---
Converting time columns...
Time column conversion attempt finished.

--- Checking Time Columns AFTER Conversion ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106578 entries, 0 to 106577
Data columns (total 3 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   order_purchase_timestamp       106578 non-null  object
 1   order_delivered_customer_date  106578 non-null  object
 2   order_estimated_delivery_date  106578 non-null  object
dtypes: object(3)
memory usage: 2.4+ MB
None

Head of time columns:
  order_purchase_timestamp order_delivered_customer_date  \
0      2017-10-02 10:56:33           2017-10-10 21:25:13   
1      2018-07-24 20:41:37           2018-08-07 15:27:45   
2      2018-08-08 08:38:49           2018-08-17 18:06:29   
3      2017-11-18 19:28:06           2017-12-02 00:28:42   
4      2018-02-13 21:18:39           2018-02-16 18:17:02   

  ord

In [81]:
# --- Process time columns (REVISED ASSIGNMENT) ---
print("Converting time columns...")
time_columns = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
conversion_successful = True
for column in time_columns:
    if column in filtered_orders.columns:
        # Ensure it's not already datetime64 before attempting conversion
        if not pd.api.types.is_datetime64_any_dtype(filtered_orders[column]):
            print(f"Attempting conversion for: {column}")
            # MODIFICATION: Use direct assignment instead of .loc
            converted_column = pd.to_datetime(filtered_orders[column], errors='coerce')
            filtered_orders[column] = converted_column # Assign back using bracket notation

            # Check results AFTER assignment
            if pd.api.types.is_datetime64_any_dtype(filtered_orders[column]):
                 print(f" -> Success: '{column}' dtype is now {filtered_orders[column].dtype}")
                 if filtered_orders[column].isna().all():
                    print(f"    ----> CRITICAL WARNING: Column '{column}' is ALL NaT values after conversion!")
                    conversion_successful = False
                 elif filtered_orders[column].isna().any():
                    print(f"    ----> NOTE: Column '{column}' has SOME NaT values after conversion.")
            else:
                 print(f" -> FAILURE: '{column}' dtype is still {filtered_orders[column].dtype}")
                 conversion_successful = False
        else:
             print(f"Column '{column}' is already datetime type.")
    else:
        print(f"Warning: Time column '{column}' not found.")
        conversion_successful = False

if conversion_successful:
    print("Time column conversion attempt finished.")
else:
    print("Time column conversion FAILED for one or more columns.")

# ***** REMOVE or COMMENT OUT the Debugging Block from the previous step *****
# print("\n--- Checking Time Columns AFTER Conversion ---")
# ... (remove the .info(), .head(), .isna() checks here) ...
# ************************************************************************

# --- Calculate Wait Times ---
# ... (rest of the code remains the same) ...

Converting time columns...
Attempting conversion for: order_purchase_timestamp
 -> Success: 'order_purchase_timestamp' dtype is now datetime64[ns]
Attempting conversion for: order_delivered_customer_date
 -> Success: 'order_delivered_customer_date' dtype is now datetime64[ns]
Attempting conversion for: order_estimated_delivery_date
 -> Success: 'order_estimated_delivery_date' dtype is now datetime64[ns]
Time column conversion attempt finished.


In [85]:
import pandas as pd
import numpy as np

# Assume filtered_orders, items, products DataFrames are loaded
# Assume haversine_distance, get_package_size, object_to_int functions are defined

# --- Calculate Distance ---
required_dist_cols = ['geolocation_lng_x', 'geolocation_lat_x', 'geolocation_lng_y', 'geolocation_lat_y']
if all(col in filtered_orders.columns for col in required_dist_cols):
    for col in required_dist_cols:
        filtered_orders[col] = pd.to_numeric(filtered_orders[col], errors='coerce')
    filtered_orders_no_nan_geo = filtered_orders.dropna(subset=required_dist_cols)
    distances = filtered_orders_no_nan_geo.apply(
        lambda row: haversine_distance(
            row["geolocation_lng_x"], row["geolocation_lat_x"],
            row["geolocation_lng_y"], row["geolocation_lat_y"]
        ),
        axis=1,
    )
    filtered_orders['distance'] = distances
    filtered_orders['distance'].fillna(0, inplace=True)
else:
    filtered_orders['distance'] = 0.0
# This line creates a new column in the filtered_orders DataFrame called "distance" by applying the haversine_distance function to calculate the distance between the seller and the customer for each row in the DataFrame.


# --- Merge Package Info ---
orders_size_weight = get_package_size(items, products)
if not orders_size_weight.empty:
    if 'product_size_cm3' in orders_size_weight.columns and 'product_weight_g' in orders_size_weight.columns:
         filtered_orders = filtered_orders.merge(orders_size_weight[['order_id', 'product_size_cm3', 'product_weight_g']], on='order_id', how='left')
    else:
        if 'product_size_cm3' not in filtered_orders.columns: filtered_orders['product_size_cm3'] = np.nan
        if 'product_weight_g' not in filtered_orders.columns: filtered_orders['product_weight_g'] = np.nan
else:
    if 'product_size_cm3' not in filtered_orders.columns: filtered_orders['product_size_cm3'] = np.nan
    if 'product_weight_g' not in filtered_orders.columns: filtered_orders['product_weight_g'] = np.nan
# These lines merge information about the package size and weight for each order from the items and products DataFrames into the filtered_orders DataFrame.


# --- Process time columns ---
time_columns = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for column in time_columns:
    if column in filtered_orders.columns:
        if not pd.api.types.is_datetime64_any_dtype(filtered_orders[column]):
            # Use direct assignment
            filtered_orders[column] = pd.to_datetime(filtered_orders[column], errors='coerce')
    else:
         filtered_orders[column] = pd.NaT
# These lines convert several time-related columns in the filtered_orders DataFrame from string format to datetime format using the pd.to_datetime function.


# --- Calculate Wait Times ---
if ('order_delivered_customer_date' in filtered_orders.columns and
    'order_purchase_timestamp' in filtered_orders.columns and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_delivered_customer_date']) and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_purchase_timestamp'])):
    filtered_orders["wait_time"] = (filtered_orders['order_delivered_customer_date'] - filtered_orders['order_purchase_timestamp']).dt.days
else:
    filtered_orders["wait_time"] = np.nan

if ('order_estimated_delivery_date' in filtered_orders.columns and
    'order_purchase_timestamp' in filtered_orders.columns and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_estimated_delivery_date']) and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_purchase_timestamp'])):
    filtered_orders["est_wait_time"] = (filtered_orders['order_estimated_delivery_date'] - filtered_orders['order_purchase_timestamp']).dt.days
else:
    filtered_orders["est_wait_time"] = np.nan
# These lines calculate the wait time and estimated wait time for each order in days, based on the difference between the purchase time and the delivered/estimated delivery time.


# --- Extract Date Components ---
if ('order_purchase_timestamp' in filtered_orders.columns and
    pd.api.types.is_datetime64_any_dtype(filtered_orders['order_purchase_timestamp'])):
    filtered_orders["purchase_dow"] = filtered_orders.order_purchase_timestamp.dt.dayofweek
    filtered_orders["year"] = filtered_orders.order_purchase_timestamp.dt.year
    filtered_orders["purchase_month"] = filtered_orders.order_purchase_timestamp.dt.month
else:
    filtered_orders["purchase_dow"] = np.nan
    filtered_orders["year"] = np.nan
    filtered_orders["purchase_month"] = np.nan
# These lines extract the day of the week, year, and month from the order_purchase_timestamp column in the filtered_orders DataFrame.


# --- Rename Geolocation State Columns (Necessary before final selection) ---
geo_state_rename_map = {
    'geolocation_state_x': 'geolocation_state_seller',
    'geolocation_state_y': 'geolocation_state_customer'
}
geo_cols_to_rename = {k: v for k, v in geo_state_rename_map.items() if k in filtered_orders.columns}
if geo_cols_to_rename:
    filtered_orders = filtered_orders.rename(columns=geo_cols_to_rename)


# --- Create Final DataFrame ---
final_columns_needed = [
    'purchase_dow', 'purchase_month', 'year',
    'product_size_cm3', 'product_weight_g',
    'geolocation_state_customer', # Expecting these after renaming
    'geolocation_state_seller',   # Expecting these after renaming
    'distance',
    'wait_time', 'est_wait_time'
]
for col in final_columns_needed:
    if col not in filtered_orders.columns:
        filtered_orders[col] = np.nan
final_df = filtered_orders[final_columns_needed].copy()
# This line creates a new DataFrame called final_df by selecting certain columns from the filtered_orders DataFrame.


# --- Calculate Delay Feature ---
if 'wait_time' in final_df.columns and 'est_wait_time' in final_df.columns:
    delay_diff = (final_df['wait_time'] - final_df['est_wait_time']).fillna(0)
    final_df['delay'] = (delay_diff > 0).astype(int)
else:
    final_df['delay'] = 0
# These lines calculate the delay time for each order and encode it as a binary variable indicating whether there was a delay or not.


# --- Encode Final DataFrame ---
final_df_enc = final_df.apply(lambda x: object_to_int(x), axis=0)
# This line applies the object_to_int function to each column in the final_df DataFrame to convert categorical variables into integer format. The object_to_int function is not shown in the code provided, but it likely uses the LabelEncoder class from the scikit-learn library to perform the conversion.


# --- Display Final Output ---
final_df_enc.head()

C:\Users\jashp\AppData\Local\Temp\ipykernel_9776\3744644978.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_orders['distance'].fillna(0, inplace=True)
C:\Users\jashp\AppData\Local\Temp\ipykernel_9776\1260564690.py:56: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_object_dtype(series) or pd.api.types.is_categorical_dtype(series):


Encoding categorical column: geolocation_state_customer
Encoding categorical column: geolocation_state_seller


C:\Users\jashp\AppData\Local\Temp\ipykernel_9776\1260564690.py:56: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_object_dtype(series) or pd.api.types.is_categorical_dtype(series):


,purchase_dow,purchase_month,year,product_size_cm3,product_weight_g,geolocation_state_customer,geolocation_state_seller,distance,wait_time,est_wait_time,delay
0,0,10,2017,1976.0,500.0,0,0,0.0,8,15,0
1,1,7,2018,4693.0,400.0,1,1,0.0,13,19,0
2,2,8,2018,9576.0,420.0,0,0,0.0,9,26,0
3,5,11,2017,6000.0,450.0,1,1,0.0,13,26,0
4,1,2,2018,11475.0,250.0,0,0,0.0,2,12,0


In [89]:
try:
    final_df_enc.to_csv('final_OTD_time_forecasting_dataframe.csv', index=False)
    print("\nSuccessfully exported 'final_OTD_time_forecasting_dataframe.csv'") # Success message
except Exception as e:
    print(f"\nError exporting CSV: {e}")



Successfully exported 'final_OTD_time_forecasting_dataframe.csv'
